# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""88945527298968…",1.2927e19,"""93837818692862…","""../../../../da…","""73239185242105…","""77675970219477…",false
"""11719919838601…",1.4856e19,"""17648699915268…","""../../../../da…","""73239185242105…","""11920231821231…",true
"""12160588939708…",5.6458e17,"""97281193061303…","""../../../../da…","""23624208644214…","""77675970219477…",false
"""11719919838601…",4.0723e18,"""67779136794306…","""../../../../da…","""95720002195558…","""77675970219477…",false
"""88945527298968…",1.4677e19,"""83978650070482…","""../../../../da…","""95720002195558…","""85522487177668…",true
"""12160588939708…",1.6330e19,"""14033821259909…","""../../../../da…","""56176689662753…",null,false
"""12160588939708…",8.7675e18,"""18050029038812…","""../../../../da…","""95720002195558…","""85522487177668…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""16582436097159…",1.2927e19,"""82180385016239…","""../../../../da…","""FirstMortgage3…","""10125362313690…","""Rejected"""
"""13139364891275…",1.4856e19,"""17974943830241…","""../../../../da…","""FirstMortgage3…","""81416166200102…","""Accepted"""
"""68079525977637…",5.6458e17,"""16311104902049…","""../../../../da…","""MoneyMarketSav…","""10125362313690…","""Rejected"""
"""13139364891275…",4.0723e18,"""11650568885024…","""../../../../da…","""BasicChecking""","""10125362313690…","""Rejected"""
"""16582436097159…",1.4677e19,"""16974687978395…","""../../../../da…","""BasicChecking""","""71876257215794…","""Accepted"""
"""68079525977637…",1.6330e19,"""16926878725590…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""68079525977637…",8.7675e18,"""66095262115458…","""../../../../da…","""BasicChecking""","""71876257215794…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""16686079065500…",1.2927e19,"""14959839893282…","""../../../../da…","""FirstMortgage3…","""17140456795469…","""Rejected"""
"""23747817491141…",1.4856e19,"""52912516683147…","""../../../../da…","""FirstMortgage3…","""49294037482609…","""Accepted"""
"""99074266395035…",5.6458e17,"""10515225873486…","""../../../../da…","""MoneyMarketSav…","""17140456795469…","""Rejected"""
"""23747817491141…",4.0723e18,"""91774708609900…","""../../../../da…","""BasicChecking""","""17140456795469…","""Rejected"""
"""16686079065500…",1.4677e19,"""11502310219706…","""../../../../da…","""BasicChecking""","""49098197024273…","""Accepted"""
"""99074266395035…",1.6330e19,"""91056769612006…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""99074266395035…",8.7675e18,"""87685034873596…","""../../../../da…","""BasicChecking""","""49098197024273…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""44345192573017…",1.2927e19,"""18389633021022…","""FirstMortgage3…","""16107099235534…","""Rejected"""
"""44376021020444…",1.4856e19,"""60236920906636…","""FirstMortgage3…","""96895691805128…","""Accepted"""
"""13161436144832…",5.6458e17,"""14503888285351…","""MoneyMarketSav…","""16107099235534…","""Rejected"""
"""44376021020444…",4.0723e18,"""72425077910630…","""BasicChecking""","""16107099235534…","""Rejected"""
"""44345192573017…",1.4677e19,"""16929505030521…","""BasicChecking""","""97404980987137…","""Accepted"""
"""13161436144832…",1.6330e19,"""14300843658167…","""UPlusFinPerson…",null,"""Rejected"""
"""13161436144832…",8.7675e18,"""11116172582286…","""BasicChecking""","""97404980987137…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'